In [1]:
#import libraries
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
import string
import json
import itertools 
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import staleness_of 
from selenium.webdriver.common.by import By
from contextlib import contextmanager

from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [2]:
class wait_for_page_load(object):

    def __init__(self, browser):
        self.browser = browser

    def __enter__(self):
        self.old_page = self.browser.find_element_by_tag_name('html')

    def page_has_loaded(self):
        new_page = self.browser.find_element_by_tag_name('html')
        return new_page.id != self.old_page.id

    def __exit__(self, *_):
        wait_for(self.page_has_loaded)

In [3]:
def wait_for(xpath):
    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
    except:
        return False
        print("Timed Out")
    return True

In [4]:
def stats_scraper(current_page):
    soup = BeautifulSoup(current_page, 'html.parser')
    table = soup.find_all('table')[1]
    table_rows = table.find_all('tr')

    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)
    df = pd.DataFrame(res)

    df.iat[0,0]
    clean_lists = []
    clean_lists.append(res[0][0])
    clean_lists.append(res[1][0])
    clean_lists.append(res[2][0])
    clean_lists.append(res[3][0])
    clean_lists.append(res[1][2])
    clean_lists.append(res[2][2])

    for x in range(2,len(df)):
        clean_lists.append(res[3][x])

    for x in range(1,len(df)-1):
        clean_lists.append(res[4][x])

    out = pd.DataFrame(clean_lists).transpose()

    if len(out.columns)==12:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "split_1", 10: "split_2", 11: "split_3"}, inplace=True)
    if len(out.columns)==14:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "split_1", 11: "split_2", 12: "split_3", 13: "split_4"}, inplace=True)
    if len(out.columns)==16:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "split_1", 12: "split_2", 13: "split_3", 14: "split_4", 15: "split_5"}, inplace=True)
    if len(out.columns)==18:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6",  12: "split_1", 13: "split_2", 14: "split_3", 15: "split_4", 16: "split_5", 17: "split_6"}, inplace=True)
    if len(out.columns)==20:
        out.rename(columns={0: "race_id", 1: "class", 2: "race_name", 3: "stakes", 4: "going", 5: "track_type", 6: "time_1", 7: "time_2", 8: "time_3", 9: "time_4", 10: "time_5", 11: "time_6", 12: "time_7",  13: "split_1", 14: "split_2", 15: "split_3", 16: "split_4", 17: "split_5", 18: "split_6", 19: "split_7"}, inplace=True)
    return out

In [5]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [17]:
# this cell will crawl races 1 to n 
def race_looper():
    wait_for("/html/body/div/div[2]/table")
    upper_bound = len(driver.find_elements_by_xpath("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr[1]/td"))
    df_list_race_results = []
    df_list_race_stats = []

    for race in range(2,upper_bound):
        with wait_for_page_load(driver):
            wait_for("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr[1]/td[%d]" %race)
            driver.find_element_by_xpath("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr/td[%d]" %race).click()
        
        time.sleep(3)
        wait_for("//div[@class='race_tab']/table/thead/tr[@class='bg_blue color_w font_wb']/td[1]")
        s = driver.find_element_by_xpath("//div[@class='race_tab']/table/thead/tr[@class='bg_blue color_w font_wb']/td[1]").text.strip()
        actual_race_number = re.search('(?<=\s)(\d+)(?=\s)', s).group(1)
        
        small_df_list=pd.read_html(driver.page_source)
        df_list_race_results.append(small_df_list[2])
        current_race_stats = stats_scraper(driver.page_source)
        df_list_race_stats.append(current_race_stats)
        print("Race %s" %actual_race_number)
        
    return df_list_race_results, df_list_race_stats     

SyntaxError: invalid syntax (<ipython-input-17-84905926316c>, line 5)

In [7]:
driver = webdriver.Firefox()
driver.get('https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx?RaceDate=2021/02/14')

In [53]:
print(driver.find_element_by_xpath("//select[@id='selectId']/option[161]").text)

18/09/2019


In [18]:
big_df_results = []
big_df_stats = []

In [54]:
for race_date_index in range(162,len(driver.find_elements_by_xpath("//select[@id='selectId']/option"))+1):
    if race_date_index == 1: continue # to skip if there is an empty date at the top
# for race_date_index in range(1,len(driver.find_elements_by_xpath("//select[@id='selectId']/option"))+1): #use this line if they dont have an empty date at the top of index
   
    
    with wait_for_page_load(driver):
        wait_for("//select[@id='selectId']/option[%d]" %race_date_index)
        driver.find_element_by_xpath("//select[@id='selectId']/option[%d]" %race_date_index).click()
        print(driver.find_element_by_xpath("//select[@id='selectId']/option[%d]" %race_date_index).text)  
    time.sleep(5)    
    
    with wait_for_page_load(driver):    
        wait_for("//div[3]/p[1]/span[2]/a/img")
        driver.find_element_by_xpath("//div[3]/p[1]/span[2]/a/img").click()
    time.sleep(5)
    
    if check_exists_by_xpath("//select[@id='selectId']/option[%d]" %race_date_index):
        if driver.find_element_by_xpath("//select[@id='selectId']/option[%d]" %race_date_index).text not in driver.find_element_by_xpath("//p[@class='f_clear']/span[@class='f_fl f_fs13']").text: 
            with wait_for_page_load(driver):
                driver.find_element_by_xpath("//div[3]/p[1]/span[2]/a/img").click()
                time.sleep(5)
            
    
    # skip if track is overseas
    if "overseas" in driver.current_url:
        with wait_for_page_load(driver):
            driver.execute_script("window.history.go(-1)")
            time.sleep(10)
            continue
    
    # skip if any races are overseas
    if len(driver.find_elements_by_xpath("//table[@class='f_fs12 f_fr js_racecard']/tbody/tr")) != 1: 
        continue
                    
    df_list_race_results, df_list_race_stats = race_looper()    
    big_df_results.append(df_list_race_results)
    big_df_stats.append(df_list_race_stats)

15/09/2019
11/09/2019
Race 1
Race 2
Race 3
Race 4
Race 5
Race 6
Race 7
Race 8
08/09/2019
01/09/2019
Race 1
Race 2
Race 3
Race 4
Race 5
Race 6
Race 7
Race 8
Race 9
Race 10


In [79]:
final_df_results = pd.DataFrame()

for df in itertools.chain.from_iterable(big_df_results):
    final_df_results = final_df_results.append(df)

In [80]:
final_df_results

,Pla.,Horse No.,Horse,Jockey,Trainer,Act. Wt.,Declar. Horse Wt.,Dr.,LBW,RunningPosition,Finish Time,Win Odds
0,1,3.0,KUNGFUMASTER PANDA(C153),K C Leung,P O'Sullivan,132.0,1100,8,-,11 9 10 9 1,2:03.68,4.7
1,2,5.0,AMAZING AGILITY(V192),K Teetan,L Ho,129.0,985,1,1-1/4,4 5 6 5 2,2:03.89,5.2
2,3,14.0,THE JOY OF GIVING(A249),V Borges,C W Chang,113.0,1012,12,1-3/4,12 12 13 14 3,2:03.98,53
3,4,11.0,FORTUNE PATROL(A245),M F Poon,A T Millard,116.0,1037,2,2-1/4,7 7 8 6 4,2:04.03,5.1
4,5,8.0,JOLLY FOREVER(D331),H N Wong,W Y So,120.0,1086,10,3,13 13 12 12 5,2:04.15,65
...,...,...,...,...,...,...,...,...,...,...,...,...
9,10,8.0,BEAUTY ENERGY(B149),K Teetan,J Moore,121.0,1073,8,5,2 2 3 10,1:22.38,6.3
10,11,3.0,PARTY TOGETHER(C210),N Callan,L Ho,129.0,1035,11,5-1/2,9 9 11 11,1:22.47,285
11,12,2.0,CHUNG WAH SPIRIT(A168),T H So,C H Yip,128.0,1090,9,7-1/4,13 14 13 12,1:22.76,199
12,13,12.0,JOLLY CONVERGENCE(V293),L Hewitson,R Gibson,118.0,1074,12,9,14 13 14 13,1:23.04,175


In [83]:
final_df_stats = pd.DataFrame()

for df in itertools.chain.from_iterable(big_df_stats):
    final_df_stats = final_df_stats.append(df)

In [84]:
final_df_stats

,race_id,class,race_name,stakes,going,track_type,time_1,time_2,time_3,split_1,split_2,split_3
0,RACE 1 (421),Class 5 - 2000M - (40-0),YUE YEE HANDICAP,"HK$ 750,000",GOOD,"TURF - ""A"" Course",(25.44),(49.62),(1:14.86),25.44,24.18,25.24
0,RACE 2 (422),Class 5 - 1400M - (40-0),RED PACKET HANDICAP,"HK$ 750,000",GOOD,"TURF - ""A"" Course",(13.88),(36.39),(1:00.87),13.88,22.51,24.48
0,RACE 3 (423),Class 4 - 1400M - (60-40),LUCKY STAR HANDICAP,"HK$ 1,000,000",GOOD,"TURF - ""A"" Course",(13.53),(35.45),(58.83),13.53,21.92,23.38
0,RACE 4 (424),Class 4 - 1200M - (60-40),KUT CHEONG HANDICAP,"HK$ 1,000,000",GOOD,"TURF - ""A"" Course",(24.06),(47.24),(1:10.19),24.06,23.18,22.95
0,RACE 5 (425),Class 4 - 1200M - (60-40),KUT CHEONG HANDICAP,"HK$ 1,000,000",GOOD,"TURF - ""A"" Course",(24.29),(46.60),(1:09.73),24.29,22.31,23.13
...,...,...,...,...,...,...,...,...,...,...,...,...
0,RACE 6 (6),Class 4 - 1000M - (60-40),YI TUNG SHAN HANDICAP,"HK$ 967,000",GOOD,"TURF - ""B"" Course",(13.32),(33.96),(57.74),13.32,20.64,23.78
0,RACE 7 (7),Class 3 - 1200M - (80-60),SUNSET PEAK HANDICAP,"HK$ 1,450,000",GOOD,"TURF - ""B"" Course",(23.79),(45.80),(1:08.47),23.79,22.01,22.67
0,RACE 8 (8),Class 2 - 1400M - (100-80),TAI MO SHAN HANDICAP,"HK$ 2,100,000",GOOD,"TURF - ""B"" Course",(13.48),(35.06),(57.88),13.48,21.58,22.82
0,RACE 9 (9),Class 3 - 1000M - (80-60),LANTAU PEAK HANDICAP,"HK$ 1,450,000",GOOD,"TURF - ""B"" Course",(13.35),(33.88),(56.91),13.35,20.53,23.03
